In [1]:
import torch

class RNNModel(torch.nn.Module):
    def __init__(self, hidden_size, dict_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.dict_size = dict_size
        self.embeddings = torch.nn.Embedding(dict_size, hidden_size)
        self.wh = torch.nn.Linear(hidden_size, hidden_size, bias=False)
        self.wy = torch.nn.Linear(hidden_size, hidden_size, bias=False)
        self.uh = torch.nn.Linear(hidden_size, hidden_size, bias=False)
        self.bh = torch.nn.Parameter(torch.randn(hidden_size))
        self.by = torch.nn.Parameter(torch.randn(hidden_size))
        self.projection = torch.nn.Linear(hidden_size, dict_size)

    def forward(self, x, h):
        x = self.embeddings(torch.tensor([x]))
        h = torch.sigmoid(self.wh(x) + self.uh(h) + self.bh)
        y = self.projection(torch.sigmoid(self.wy(h) + self.by))
        return y, h

    def zero_state(self):
        return torch.zeros(self.hidden_size)

In [2]:
import requests

all_shakespeare = requests.get("https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt").content.decode()
print(len(all_shakespeare))

5458199


In [3]:
dictionary = list(set(all_shakespeare))
dictionary.append("<start>")
dictionary.append("<end>")
print(dictionary)
print(len(dictionary))

sym2idx = {s: i for i, s in enumerate(dictionary)}
print(sym2idx)

['?', ',', 'K', 'I', '*', 'L', ')', 'H', 'm', '7', 'd', '.', 'o', 'a', ':', '6', '1', 'y', ']', 'f', 'u', 'e', '8', 'W', 'Z', '_', 'k', '4', 'b', 'J', 'U', 'O', 'q', 'i', 'p', 'S', '`', '"', 'n', 'V', 'r', 'C', '[', 'j', 'X', 'w', 'B', '3', '=', 'E', 'v', '!', ' ', 'l', 'F', '/', ';', 'Y', 'g', '0', '<', '|', 'z', 'h', '%', 'D', '&', 'R', 't', 's', '#', 'Q', '(', '2', '}', "'", 'x', 'M', '>', 'P', 'T', 'N', '5', 'A', 'G', 'c', '-', '@', '~', '\n', '9', '<start>', '<end>']
93
{'?': 0, ',': 1, 'K': 2, 'I': 3, '*': 4, 'L': 5, ')': 6, 'H': 7, 'm': 8, '7': 9, 'd': 10, '.': 11, 'o': 12, 'a': 13, ':': 14, '6': 15, '1': 16, 'y': 17, ']': 18, 'f': 19, 'u': 20, 'e': 21, '8': 22, 'W': 23, 'Z': 24, '_': 25, 'k': 26, '4': 27, 'b': 28, 'J': 29, 'U': 30, 'O': 31, 'q': 32, 'i': 33, 'p': 34, 'S': 35, '`': 36, '"': 37, 'n': 38, 'V': 39, 'r': 40, 'C': 41, '[': 42, 'j': 43, 'X': 44, 'w': 45, 'B': 46, '3': 47, '=': 48, 'E': 49, 'v': 50, '!': 51, ' ': 52, 'l': 53, 'F': 54, '/': 55, ';': 56, 'Y': 57, 'g': 58

In [4]:
model = RNNModel(128, len(dictionary))
for param in model.parameters():
    print(param)

Parameter containing:
tensor([-1.9266e+00,  3.7460e-01, -8.4192e-01,  5.7829e-01,  1.5031e-01,
        -7.6483e-01,  1.3104e+00, -7.5613e-01, -1.3940e+00,  9.4137e-01,
         4.6848e-01,  4.9371e-01,  9.7709e-01, -8.1185e-01, -3.9525e-02,
         1.2366e+00, -1.6081e-01,  9.1398e-02, -9.4032e-01, -1.7903e-02,
         3.1462e-01,  5.3082e-01,  5.8745e-01,  8.0033e-01, -6.9968e-01,
        -9.3662e-01,  7.6308e-02, -5.8648e-01, -2.3810e+00, -3.1006e-01,
         9.5466e-02, -2.7594e+00, -1.3723e+00,  3.0150e-01,  8.1697e-01,
        -5.8202e-01,  7.6177e-01, -1.3148e+00, -1.1513e-02,  6.1183e-01,
         8.3423e-02, -8.2471e-01, -6.9657e-01,  8.7332e-01,  1.2039e+00,
         3.5023e-01,  1.7394e+00,  2.2808e-03,  2.0652e+00,  4.5765e-01,
        -1.6390e-01,  4.2148e-01,  7.1645e-01,  5.6960e-01, -1.8987e+00,
        -2.8255e-01, -1.6632e-01, -2.3588e+00,  5.7328e-01,  3.2845e-01,
        -1.0897e-01,  1.3576e+00, -3.9493e-01, -1.3344e+00, -1.1869e+00,
         6.4032e-01,  9.7548e

In [5]:
import random

random.seed(42)

data = all_shakespeare.split("\n\n")
data = list(filter(lambda x: x, data))
random.shuffle(data)

print(len(data))
print(data[128])

6483
  SICINIUS. Well, here he comes.
  MENENIUS. Calmly, I do beseech you.
  CORIOLANUS. Ay, as an ostler, that for th' poorest piece
    Will bear the knave by th' volume. Th' honour'd gods
    Keep Rome in safety, and the chairs of justice
    Supplied with worthy men! plant love among's!
    Throng our large temples with the shows of peace,
    And not our streets with war!
  FIRST SENATOR. Amen, amen!
  MENENIUS. A noble wish.


In [6]:
train = [data[i] for i in range(len(data)) if i % 10 != 0]
test = [data[i] for i in range(len(data)) if i % 10 == 0]

print(train[-1])
print("")
print(test[-1])

  TRANIO. Gentlemen, God save you! If I may be bold,
    Tell me, I beseech you, which is the readiest way
    To the house of Signior Baptista Minola?
  BIONDELLO. He that has the two fair daughters; is't he you mean?
  TRANIO. Even he, Biondello.
  GREMIO. Hark you, sir, you mean not her to-
  TRANIO. Perhaps him and her, sir; what have you to do?
  PETRUCHIO. Not her that chides, sir, at any hand, I pray.
  TRANIO. I love no chiders, sir. Biondello, let's away.
  LUCENTIO.  [Aside]  Well begun, Tranio.
  HORTENSIO. Sir, a word ere you go.
    Are you a suitor to the maid you talk of, yea or no?
  TRANIO. And if I be, sir, is it any offence?
  GREMIO. No; if without more words you will get you hence.  
  TRANIO. Why, sir, I pray, are not the streets as free
    For me as for you?
  GREMIO. But so is not she.

ALLS WELL THAT ENDS WELL


In [7]:
import tqdm

def train_epoch(data, model):
    model.train()
    loss_function = torch.nn.CrossEntropyLoss()
    # optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    optimizer = torch.optim.AdamW(model.parameters())
    total_loss, total_count = 0.0, 1e-38
    for item in tqdm.tqdm(data):
        optimizer.zero_grad()
        outputs = []
        answers = torch.tensor([sym2idx[sym] for sym in list(item) + ["<end>"]])
        state = model.zero_state()
        for i, x in enumerate(["<start>"] + list(item)):
            y, state = model(sym2idx[x], state)
            outputs.append(y)
        outputs = torch.cat(outputs)
        #print(outputs.shape)
        #print(answers.shape)
        #print(outputs)
        loss = loss_function(outputs, answers)
        total_loss += loss.item()
        total_count += 1
        loss.backward()
        optimizer.step()
    return total_loss / total_count

def test_epoch(data, model):
    model.eval()
    loss_function = torch.nn.CrossEntropyLoss()
    total_loss, total_count = 0.0, 1e-38
    for item in tqdm.tqdm(data):
        outputs = []
        answers = torch.tensor([sym2idx[sym] for sym in list(item) + ["<end>"]])
        state = model.zero_state()
        for i, x in enumerate(["<start>"] + list(item)):
            y, state = model(sym2idx[x], state)
            outputs.append(y)
        outputs = torch.cat(outputs)
        loss = loss_function(outputs, answers)
        total_loss += loss.item()
        total_count += 1
    return total_loss / total_count

for _ in range(10):
    train_loss = train_epoch(train, model)
    test_loss = test_epoch(test, model)
    print("Epoch loss: {:.5f} {:.5f}".format(train_loss, test_loss))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 649/649 [01:40<00:00,  6.47it/s]


Epoch loss: 2.09608 1.79229


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 649/649 [01:40<00:00,  6.46it/s]


Epoch loss: 1.71768 1.65485


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 649/649 [01:40<00:00,  6.49it/s]


...

Epoch loss: 1.39086 1.40252


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 649/649 [01:59<00:00,  5.42it/s]


Epoch loss: 1.36880 1.38364


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 649/649 [01:40<00:00,  6.44it/s]

Epoch loss: 1.34822 1.36728
